# Calibrate aerosol forcing

Rather than take the individual forcers, we'll take the Smith et al. (2021) approach of curve fitting precursors.

In [ ]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

import matplotlib.pyplot as pl
import pandas as pd
import pooch
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
climate_response_df

In [ ]:
aerosol_df = pd.read_csv('../data/smith2023/ukesm_ghan2013_aerosol_forcing.csv', index_col=0)

In [ ]:
erfari = aerosol_df['ERFari']

In [ ]:
erfaci = aerosol_df['ERFaci']

In [ ]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
species = ['BC', 'OC', 'Sulfur', 'Aerosol-cloud interactions']
species, properties = read_properties(species=species)

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
bc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'1850':'2014'].values.squeeze()
oc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'1850':'2014'].values.squeeze()
so2 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'1850':'2014'].values.squeeze()

In [ ]:
def erfaci_rootfinder(x, shape_so2, shape_bc, shape_oc, beta):
    f = FAIR(temperature_prescribed=True)
    f.define_time(1850, 2015, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(species, properties)
    f.allocate()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    f.fill_species_configs()
    f.species_configs['baseline_emissions'].loc[dict(specie='Sulfur')] = 4.5444637755469
    f.species_configs['baseline_emissions'].loc[dict(specie='BC')] = 2.57112447874271
    f.species_configs['baseline_emissions'].loc[dict(specie='OC')] = 18.2268247866406
    
    # fill emissions and concentrations
    f.emissions.loc[dict(specie='Sulfur')] = x[0][:,None,None]
    f.emissions.loc[dict(specie='BC')] = x[1][:,None,None]
    f.emissions.loc[dict(specie='OC')] = x[2][:,None,None]

    f.temperature[:] = 0

    # these things we want to vary
#    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = alpha_so2
#    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = alpha_bc
#    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = alpha_oc
    f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = shape_so2
    f.species_configs['aci_shape'].loc[dict(specie='BC')] = shape_bc
    f.species_configs['aci_shape'].loc[dict(specie='OC')] = shape_oc
    f.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = beta
    
    f.run(progress=False)
    out = 0.5*(f.forcing.values[1:, 0, 0, 3] + f.forcing.values[:-1, 0, 0, 3])
    return out

In [ ]:
solution = curve_fit(erfaci_rootfinder, [so2, bc, oc], erfaci, bounds=((0, 0, 0, -np.inf), (np.inf, np.inf, np.inf, np.inf)))

In [ ]:
solution

In [ ]:
species = ['BC', 'OC', 'Sulfur', 'Aerosol-radiation interactions']
species, properties = read_properties(species=species)

def erfari_rootfinder(x, alpha_so2, alpha_bc, alpha_oc):
    f = FAIR(temperature_prescribed=True)
    f.define_time(1850, 2015, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(species, properties)
    f.allocate()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    f.fill_species_configs()
    f.species_configs['baseline_emissions'].loc[dict(specie='Sulfur')] = 4.5444637755469
    f.species_configs['baseline_emissions'].loc[dict(specie='BC')] = 2.57112447874271
    f.species_configs['baseline_emissions'].loc[dict(specie='OC')] = 18.2268247866406
    
    # fill emissions and concentrations
    f.emissions.loc[dict(specie='Sulfur')] = x[0][:,None,None]
    f.emissions.loc[dict(specie='BC')] = x[1][:,None,None]
    f.emissions.loc[dict(specie='OC')] = x[2][:,None,None]

    f.temperature[:] = 0

    # these things we want to vary
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = alpha_so2
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = alpha_bc
    f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = alpha_oc
    
    f.run(progress=False)
    out = 0.5*(f.forcing.values[1:, 0, 0, 3] + f.forcing.values[:-1, 0, 0, 3])
    return out

In [ ]:
solution = curve_fit(erfari_rootfinder, [so2, bc, oc], erfari)

In [ ]:
solution